In [2]:
import pandas as pd
import os.path
import numpy as np
import numba as nb
import gc

In [5]:
file_path = 'Fixed Census v3.csv'
original_path = '98-401-X2021006_English_CSV_data_Ontario.csv'

In [4]:
# Helper function to save the data from RAM to Disk
def append_to_file(file_path, to_save_df):
    header = True
    # if file exists then don't append the header again
    if os.path.isfile(file_path):
        header = False
        
    to_save_df.to_csv(file_path, mode='a', header=header, index=False)
    
chunk_size = 256_961
chunk_iter = 896

In [9]:
use_cols = ["ALT_GEO_CODE", "CHARACTERISTIC_NAME", "C1_COUNT_TOTAL", "GEO_NAME"]
df = pd.read_csv(original_path, encoding='latin1', nrows=3000, low_memory=True, usecols=use_cols)

# Gets the labels for the new data frame
labels = tuple(df.loc[0:2631, "CHARACTERISTIC_NAME"].tolist())

del df

print ("Deleted Df")
count = 0       

@nb.jit
def process_row(temp_df, new_df):
    for temp_row in temp_df.itertuples(index=False):
        new_df.loc[new_df['ALT_GEO_CODE'] == temp_row.ALT_GEO_CODE, temp_row.CHARACTERISTIC_NAME] = temp_row.C1_COUNT_TOTAL
        new_df.loc[new_df['ALT_GEO_CODE'] == temp_row.ALT_GEO_CODE, 'GEO_NAME'] = temp_row.GEO_NAME        


for df in pd.read_csv(original_path, encoding='latin1', chunksize=26_310, low_memory=True, usecols=use_cols):
    count += 1
    print(f"Loaded Chunk: {count}")
    
    new_df = pd.DataFrame(columns=labels)
    new_df['ALT_GEO_CODE'] = df['ALT_GEO_CODE'].unique()
    for row in new_df.itertuples(index=False):
        
        temp_df = df.groupby('ALT_GEO_CODE').get_group(row.ALT_GEO_CODE)
        
        process_row(temp_df, new_df)
            
    append_to_file(file_path, new_df
    
    del new_df, temp_df, df
    gc.collect()

Deleted Df
Loaded Chunk: 1


C:\Users\shann\AppData\Local\Temp\ipykernel_17004\3710910953.py:12: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @nb.jit
C:\Users\shann\AppData\Local\Temp\ipykernel_17004\3710910953.py:12: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "process_row" failed type inference due to: non-precise type pyobject
During: typing of argument at C:\Users\shann\AppData\Local\Temp\ipykernel_17004\3710910953.py (12)

File "..\..\..\AppData\Local\Temp\ipykernel_17004\3710910953.py", line 12:
<source missing, REPL/exec in use?>

  @nb.jit
C:\Users\shann\AppData\Local\Temp\ipykernel_17004\3710910953.py:12: NumbaWarning: 
Compilation is fa

Loaded Chunk: 2
Loaded Chunk: 3
Loaded Chunk: 4
Loaded Chunk: 5
Loaded Chunk: 6
Loaded Chunk: 7
Loaded Chunk: 8
Loaded Chunk: 9
Loaded Chunk: 10
Loaded Chunk: 11
Loaded Chunk: 12
Loaded Chunk: 13
Loaded Chunk: 14
Loaded Chunk: 15
Loaded Chunk: 16
Loaded Chunk: 17
Loaded Chunk: 18
Loaded Chunk: 19
Loaded Chunk: 20
Loaded Chunk: 21
Loaded Chunk: 22
Loaded Chunk: 23
Loaded Chunk: 24
Loaded Chunk: 25
Loaded Chunk: 26
Loaded Chunk: 27
Loaded Chunk: 28
Loaded Chunk: 29
Loaded Chunk: 30
Loaded Chunk: 31
Loaded Chunk: 32
Loaded Chunk: 33
Loaded Chunk: 34
Loaded Chunk: 35
Loaded Chunk: 36
Loaded Chunk: 37
Loaded Chunk: 38
Loaded Chunk: 39
Loaded Chunk: 40
Loaded Chunk: 41
Loaded Chunk: 42
Loaded Chunk: 43
Loaded Chunk: 44
Loaded Chunk: 45
Loaded Chunk: 46
Loaded Chunk: 47
Loaded Chunk: 48
Loaded Chunk: 49
Loaded Chunk: 50
Loaded Chunk: 51
Loaded Chunk: 52
Loaded Chunk: 53
Loaded Chunk: 54
Loaded Chunk: 55
Loaded Chunk: 56
Loaded Chunk: 57
Loaded Chunk: 58
Loaded Chunk: 59
Loaded Chunk: 60
Loade

In [7]:
fixed_df = pd.read_csv(file_path, encoding='latin1', low_memory=True)


C:\Users\shann\AppData\Local\Temp\ipykernel_2956\3936286497.py:1: DtypeWarning: Columns (2633) have mixed types. Specify dtype option on import or set low_memory=False.
  fixed_df = pd.read_csv(file_path, encoding='latin1', low_memory=True)


In [15]:
pd.DataFrame(fixed_df.columns).to_csv('columns.csv', index=False)

In [14]:
pd.DataFrame(fixed_df['GEO_NAME'].str.strip().unique()).to_csv('geo_names.csv', index=False)